In [1]:
%load_ext autoreload
%autoreload 2
%aimport
%matplotlib inline

Modules to reload:
all-except-skipped

Modules to skip:



In [2]:
import os
import sys
nb_dir = os.path.dirname(os.path.split(os.getcwd())[0])
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [3]:
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from turicreate import SFrame, load_sframe
from pathlib import Path
import turicreate.aggregate as agg
import numpy as np

In [4]:
from ScienceDynamics.datasets.sjr import SJR
from ScienceDynamics.config.configs import DATASETS_SJR_DIR

In [5]:
import json
import os
import matplotlib.pyplot as plt
import pandas as pd
import math
import glob
import ntpath
from tqdm import tqdm
import re

import datetime
from matplotlib.backends.backend_pdf import PdfPages
from  matplotlib.ticker import FuncFormatter

## Draw Graphs

In [6]:
import seaborn as sns
from  matplotlib.ticker import FuncFormatter

sns.set(font_scale=1.3)
plt.figure(figsize=(16, 12))
# sns.set(style="ticks")
sns.set()
def create_gird(df, col, hue,x,y,sharey=True, legend=False):

    # Initialize a grid of plots with an Axes for each walk
    grid = sns.FacetGrid(df, col=col, hue=hue, palette=sns.color_palette("hls", 4),sharey=sharey,
                         col_wrap=3, height=4.5)
    plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)))

    # Draw a horizontal line to show the starting point
    grid.map(plt.axhline, y=0, ls=":", c=".5")
#     grid.set_titles(col_template = '{col_name}')
    # Draw a line plot to show the trajectory of each random walk
    grid.map(plt.plot, x, y)
    grid.set_titles("{col_name}")
    if legend:
        grid.add_legend()


    # Adjust the arrangement of the plots
    grid.fig.tight_layout(w_pad=1)
    return grid

<Figure size 1152x864 with 0 Axes>

In [7]:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i + n]


## Analysis

### Load Data

In [8]:
spothlight = ["SARS","MERS Coronavirus", "Avian Influenza","Ebola", "Influenza", "HIV/AIDS","Hepatitis B","Hepatitis C", "Swine Flu"]

In [9]:
diseases_id= load_sframe("Data/diseases_list.csv")
disease_names = SFrame.read_csv("Data/disease_names.csv")

Finished parsing file /storage/homedir/dima/Projects/Giga/ScienceDynamics/examples/Coronavirus/Data/diseases_list.csv

Parsing completed. Parsed 9 lines in 0.028958 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /storage/homedir/dima/Projects/Giga/ScienceDynamics/examples/Coronavirus/Data/diseases_list.csv

Parsing completed. Parsed 9 lines in 0.024191 secs.

Finished parsing file /storage/homedir/dima/Projects/Giga/ScienceDynamics/examples/Coronavirus/Data/disease_names.csv

Parsing completed. Parsed 9 lines in 0.025884 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /storage/homedir/dima/Projects/Giga/ScienceDynamics/examples/Coronavirus/Data/disease_names.csv

Parsing completed. Parsed 9 lines in 0.025319 secs.

In [10]:
disease_names = disease_names.rename({"disease":"Disease"})

##### SJR

In [11]:
sjr = SJR(DATASETS_SJR_DIR)

In [12]:
sjr_sf = sjr.data

Convert SJR to float form string

In [13]:
sjr_sf["SJR"] = sjr_sf["SJR"].apply(lambda x: x.replace(",","."))
sjr_sf["SJR"] = sjr_sf["SJR"].apply(lambda x: x if x else "-1")
sjr_sf["SJR"] = sjr_sf["SJR"].astype(float)

Using default 12 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 24)"

Note that increasing the degree of parallelism also increases the memory footprint.

In [14]:
sjr_sf[sjr_sf["Sourceid"]==22471]

Rank,Sourceid,Title,Type,SJR,SJR Best Quartile,H index,Total Docs.
18031,22471,The Lancet InfectiousDiseases ...,journal,-1.0,-,201,95
4822,22471,The Lancet InfectiousDiseases ...,journal,0.5130000114440918,Q2,201,229
2212,22471,The Lancet InfectiousDiseases ...,journal,1.0369999408721924,Q1,201,253
1335,22471,The Lancet InfectiousDiseases ...,journal,1.468999981880188,Q1,201,243
1193,22471,The Lancet InfectiousDiseases ...,journal,1.6549999713897705,Q1,201,215
1225,22471,The Lancet InfectiousDiseases ...,journal,1.694000005722046,Q1,201,179
957,22471,The Lancet InfectiousDiseases ...,journal,2.0380001068115234,Q1,201,172
455,22471,The Lancet InfectiousDiseases ...,journal,3.065999984741211,Q1,201,164
259,22471,The Lancet InfectiousDiseases ...,journal,4.395999908447266,Q1,201,164
161,22471,The Lancet InfectiousDiseases ...,journal,6.376999855041504,Q1,201,198


##### Pubmed

In [23]:
diseases_pubmed = load_sframe("Data/pubmed/diseases_pubmed.sframe")

### Journals Current State

Format ISSN similarly to PubMed dataset

In [25]:
diseases_pubmed["issn_linking"]

dtype: str
Rows: 1524782
['0033-2240', '1124-0490', '0022-538X', '0001-723X', '1932-6203', '0896-8608', '1756-3305', '0022-1899', '0030-6002', '1300-4948', '0070-4113', '1386-6532', '1040-8703', '0379-5284', '0891-3668', '1932-6203', '1573-4064', '1464-2662', '1042-0533', '0026-6620', '1205-9838', '1473-3099', '1386-5056', '0042-8825', '', '1525-4135', '0022-3476', '0049-6804', '0028-0836', '0491-6514', '0016-9560', '0022-1899', '0264-410X', '0039-2499', '0302-766X', '1936-2625', '0002-9637', '0049-4755', '', '0028-3878', '1081-1710', '0022-538X', '1932-6203', '2076-3271', '0022-1899', '1043-6596', '0022-538X', '0884-0431', '1078-8956', '1553-7366', '0001-5458', '0003-4894', '0264-410X', '2168-8184', '2214-2509', '0891-1150', '0028-8446', '2093-4777', '0006-3002', '1198-743X', '0172-8172', '1867-1071', '0381-6605', '0377-1237', '0012-7183', '0034-7280', '0304-8608', '0026-9263', '2328-8957', '1735-1995', '0264-410X', '1556-679X', '0002-9637', '0021-5295', '0034-1193', '0300-5410', '003

In [26]:
diseases_pubmed["ISSN"] = diseases_pubmed["issn_linking"].apply(lambda x: x.replace("-", ""))

SJR is publushed in a delay of two years, to deal with it we correct the SJR date to its correct publication date. 

In [31]:
sjr_sf["Year"] = sjr_sf["Year"]+2

In [41]:
diseases_pubmed = diseases_pubmed.rename({"disease":"Disease"})

In [33]:
diseases_pubmed = diseases_pubmed.filter_by(spothlight,"Disease")

In [43]:
pubmed_sjr = sjr_sf.join(diseases_pubmed, on={"ISSN":"ISSN","Year":"year"})

In [44]:
pubmed_sjr = pubmed_sjr.filter_by(spothlight,"Disease")

In [45]:
pubmed_sjr=pubmed_sjr.unique()

In [46]:
pubmed_sjr["Categories Clean"] = pubmed_sjr["Categories"].apply(lambda x: re.compile(r"\s\(.*?\);{0,1}").split(x)[:-1])

In [47]:
pubmed_sjr["Categories Clean"] = pubmed_sjr["Categories Clean"].apply(lambda x: {c.strip() for c in x})

In [48]:
cat = {"Immunology", "Epidemiology", "Infectious Diseases", "Virology", "Microbiology"}

In [49]:
pubmed_sjr = pubmed_sjr[pubmed_sjr["Categories Clean"].apply(lambda x: len(set(x) & cat)>0)]

In [50]:
journal_disease = pubmed_sjr.groupby(["ISSN","Disease","Year"], operations={"Count": agg.COUNT_DISTINCT("pmid")})

In [51]:
journal_map = pubmed_sjr[["SJR" ,"Title", "ISSN","Year"]].unique()

In [52]:
journal_disease = journal_disease.join(journal_map,["ISSN","Year"])

In [53]:
journal_disease = journal_disease[journal_disease["SJR"]>0]

In [54]:
journal_disease_df = journal_disease.to_dataframe()

### Top Journals by Publications

In [55]:
journal_disease_sf = SFrame(journal_disease_df)

In [56]:
journal_disease_sf["Score"] = journal_disease_sf["Count"]*journal_disease_sf["SJR"]

In [57]:
journal_disease_score = journal_disease_sf.groupby("Disease",{"Count":agg.SUM("Count"), "Score":agg.SUM("Score")})

In [58]:
journal_disease_score["JScore"] = journal_disease_score["Score"] /journal_disease_score["Count"] 

In [59]:
journal_disease_score.sort("JScore",False)

Disease,Count,Score,JScore
Ebola,1375,4471.485005371273,3.251989094815471
MERS Coronavirus,355,842.6880009099841,2.3737690166478425
HIV/AIDS,53812,108128.65316729248,2.0093780786310207
Influenza,13386,26737.2220434919,1.9974019156948977
SARS,1060,1932.786006823182,1.8233830253048888
Swine Flu,2570,4559.577994965017,1.774154861854092
Hepatitis C,9064,15397.97101661563,1.6988052754430305
Avian Influenza,1708,2812.1620041802526,1.6464648736418341
Hepatitis B,6907,9829.149002812803,1.423070653368004


In [60]:
journal_disease_score.save("output/Journals/avg_jscore.csv")

In [61]:
journal_disease_year_score = journal_disease_sf.groupby(["Disease","Year"],{"Count":agg.SUM("Count"), "Score":agg.SUM("Score")})

In [62]:
journal_disease_year_score["JScore"] = journal_disease_year_score["Score"] /journal_disease_year_score["Count"] 

In [63]:
journal_disease_year_score.save("output/Journals/journal_disease_year_score.csv","csv")

In [64]:
journal_disease_year_score = journal_disease_year_score.sort(["Disease", "Year"])

In [88]:

sns.set(font_scale=1.3)


plt.rc('text', usetex=False)
create_gird(journal_disease_year_score[journal_disease_year_score["Count"]>5].to_dataframe(),"Disease","Disease","Year", "JScore")
plt.savefig('output/Journals/jscore.png')
plt.close()



In [66]:
sjr_sf["Categories Clean"] = sjr_sf["Categories"].apply(lambda x: re.compile(r"\s\(.*?\);{0,1}").split(x)[:-1])
sjr_sf["Categories Clean"] = sjr_sf["Categories Clean"].apply(lambda x: {c.strip() for c in x})

In [67]:
sjr_sf = sjr_sf[sjr_sf["Categories Clean"].apply(lambda x: len(set(x) & cat)>0)]

In [68]:
jtop_10 = SFrame()
for year in range(2002,2019):
    jtop_10 = jtop_10.append(sjr_sf.filter_by(year, "Year").sort('Rank')[:10])
jtop_3 = SFrame()
for year in range(2002,2019):
    jtop_3 = jtop_3.append(sjr_sf.filter_by(year, "Year").sort('Rank')[:3])

In [70]:
papers_sjr_top_10 = pubmed_sjr.join( jtop_10[["Year","Sourceid"]], on=["Year","Sourceid"])


In [71]:
papers_sjr_top_10 = pubmed_sjr.join( jtop_10[["Year","Sourceid"]], on=["Year","Sourceid"])
papers_top_10= papers_sjr_top_10.groupby(["Disease","Year"], {"Count":agg.COUNT()})
papers_top_10 = papers_top_10.sort(["Disease","Year"])

In [72]:
papers_top_10["# of Papaers in Top-10"] = papers_top_10["Count"]

In [74]:

plt.rc('text', usetex=False)
create_gird(papers_top_10.to_dataframe(),"Disease","Disease","Year", "# of Papaers in Top-10", False, False)
plt.savefig('output/Journals/top10.png')
plt.close()



In [75]:
papers_sjr_top_3 = pubmed_sjr.join( jtop_3[["Year","Sourceid"]], on=["Year","Sourceid"])
papers_top_3= papers_sjr_top_3.groupby(["Disease","Year"], {"Count":agg.COUNT()})
papers_top_3 = papers_top_3.sort(["Disease","Year"])

In [79]:


# if LaTeX is not installed or error caught, change to `usetex=False`
plt.rc('text', usetex=False)
plt.figure(figsize=(8, 6))
create_gird(papers_top_3.to_dataframe(),"Disease","Disease","Year", "Count", True, False)
plt.savefig("output/Journals/top3.png")
plt.close()



<Figure size 576x432 with 0 Axes>

In [80]:
papaers_q = pubmed_sjr.groupby(["Year","Disease", "SJR Best Quartile"],{"Count": agg.COUNT()})

In [81]:
papaers_q = papaers_q.sort(["Disease", "Year"])

In [82]:
papaers_q=papaers_q[papaers_q["SJR Best Quartile"]!="-"]

In [83]:


# if LaTeX is not installed or error caught, change to `usetex=False`
plt.rc('text', usetex=False)
plt.figure(figsize=(8, 6))
create_gird(papaers_q[papaers_q["SJR Best Quartile"]=="Q1"].to_dataframe(),"Disease","Disease","Year", "Count", False)
plt.savefig('output/Journals/q1.pdf')
plt.close()



<Figure size 576x432 with 0 Axes>

In [84]:
papaers_q["# of Papaers in Quartile"] = papaers_q["Count"]

In [85]:


# if LaTeX is not installed or error caught, change to `usetex=False`
plt.rc('text', usetex=False)
plt.figure(figsize=(8, 6))
create_gird(papaers_q.to_dataframe(),"Disease","SJR Best Quartile","Year", "# of Papaers in Quartile", False, True)
plt.savefig('output/Journals/q.png')
plt.close()



<Figure size 576x432 with 0 Axes>